In [1]:
!pip install -q transformers accelerate sentencepiece bitsandbytes youtube-transcript-api


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from youtube_transcript_api import YouTubeTranscriptApi
import torch


In [3]:
from huggingface_hub import login

login() # Follow the prompts to enter your Hugging Face credentials


In [4]:
!pip install -U bitsandbytes


In [5]:
!pip uninstall -y bitsandbytes
!pip install -U bitsandbytes --prefer-binary --quiet


Found existing installation: bitsandbytes 0.45.5
Uninstalling bitsandbytes-0.45.5:
  Successfully uninstalled bitsandbytes-0.45.5


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16,
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:
def get_transcript(video_url):
    try:
        video_id = video_url.split("v=")[-1]
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return " ".join([entry['text'] for entry in transcript])
    except Exception as e:
        return f"Error fetching transcript: {str(e)}"


In [8]:
def format_prompt(transcript):
    return f"""### Instruction:
Summarize the following YouTube transcript into well-structured, detailed study notes with bullet points and key takeaways.

### Input:
{transcript}

### Response:
"""


In [9]:
def generate_summary(prompt, max_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=max_tokens, do_sample=False)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [10]:
video_url = input("Enter YouTube Video URL: ")
transcript = get_transcript(video_url)

if "Error" in transcript:
    print(transcript)
else:
    print("\n✅ Transcript fetched. Summarizing with Mistral...\n")
    prompt = format_prompt(transcript)
    summary = generate_summary(prompt)
    print("📝 Summary:\n")
    print(summary.replace(prompt, ""))  # Remove prompt from response


Enter YouTube Video URL: https://www.youtube.com/watch?v=ETFTg1Za28U


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



✅ Transcript fetched. Summarizing with Mistral...



/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


📝 Summary:


**Background:**
- Hezbollah is a militant group based in Lebanon, one of the biggest armed non-state actors in the world with tens of thousands of trained soldiers and hundreds of thousands of missiles and rockets.
- Hezbollah's primary enemy is Israel, leading to thousands of attacks along the border over the past year.
- Hezbollah's origins can be traced back to the Ottoman Empire's breakup and the subsequent creation of Lebanon, which forced Shiite Muslims into a minority.
- The group's rise to power was influenced by the Syrian Civil War, the Israeli-Palestinian conflict, and Iran's Islamic Revolution.

**Origins of Hezbollah:**
1. **Ottoman Empire's Breakup:**
   - Europeans carved up the Middle East, creating Lebanon.
   - French-controlled Lebanon included mostly Maronite Christians but excluded Shiite Muslims, who were forced into a minority.
2. **Lebanon's Independence:**
   - Lebanon gained independence from France in 1943.
   - Constitution aimed to balance powe

# New Section